In [1]:
#import necessary modules

from bs4 import BeautifulSoup
import requests
import pandas as pd
from pandas import DataFrame
import numpy as np

In [2]:
# url to scrape from

wiki_url = "https://en.wikipedia.org/wiki/List_of_Formula_One_Grand_Prix_winners"

In [3]:
#quering the website

response = requests.get(wiki_url)

In [4]:
#parsing the above object with beautiful soup

soup = BeautifulSoup(response.text, 'html.parser')

In [5]:
#specify the table to use

table=soup.find('table', {"class":'sortable plainrowheaders wikitable'})

In [7]:
# since the column headers are in the tr tags, we find all tr tags for the column name row

rows = table.find_all('tr')

In [8]:
# seach for the column names, clean the escape characters, and assign to the column variable

columns = [v.text.replace('\n', '') for v in rows[0].find_all('th')]


In [9]:
# view the column variable, with the column names collected

print(columns)


['Rank', 'Country', 'Driver', 'Wins', 'Seasons active', 'First win', 'Last win']


In [10]:

# create a empty dataframe
df = pd.DataFrame()


# collect all row data, - every interation, it will append the data as a new row to the dataframe. 

for i in range(1, len(rows)): 
    tds = rows[i].find_all('td')
    if len(tds) == 6:
        values = [tds[0].text.replace('\n', ''), tds[1].text.replace('\n', "").replace('\xa0', ""), tds[2].text.replace('\n', ''),tds[3].text.replace('\n', ''),tds[4].text.replace('\n', ''),tds[5].text.replace('\n', '')]
    else: 
        values = [td.text.replace('\n', "").replace('\xa0', "") for td in tds]
    
    df = df.append(pd.Series(values),ignore_index = True)
    

df.columns = ['Rank', 'Country', 'Wins', 'Seasons active', 'First win', 'Last win']



In [11]:
# since the drivers wasnt being collected becuase it was written in the th tag, the below for loop will collect the data 
# from the drivers column and append it to the a list. 

a = []
for i in range(len(rows)):
    drivers = [v.text.replace('\n', '') for v in rows[i].find_all('th')]
    a.append(drivers)


In [12]:
#deleting the row headers from the drivers list collected, only require drivers column. 
del a[0]

In [13]:
#np.squeeze(a)

In [14]:
#clean the python list, moving all values into the list by itself, as opposed to each element being in its own array
# driversarray = [[1],[2],[3]] ---> driversarray = [1,2,3]

driversarray = []
for i in range(len(a)): 
    temp = a[i][0]
    driversarray.append(temp)


In [15]:
#adding the drivers list collected to the remainder of the table scraped.

df['drivers'] = driversarray


In [16]:
#converting it to csv for sharing

df.to_csv('winning.csv')